In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

In [8]:
pip install sklearn

  Using cached sklearn-0.0.tar.gz (1.1 kB)
     |████████████████████████████████| 7.2 MB 5.6 MB/s eta 0:00:01
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
     |████████████████████████████████| 302 kB 5.4 MB/s eta 0:00:01
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1316 sha256=82c39fb16a67d49b55c7b29e6cd9d1fe96e914143ce9e06d7cfa0992d7a96015
  Stored in directory: /Users/simonneumeyer/Library/Caches/pip/wheels/22/0b/40/fd3f795caaa1fb4c6cb738bc1f56100be1e57da95849bfc897
Successfully built sklearn
Note: you may need to restart the kernel to use updated packages.


### Load data

We load each comparison group dataset as a different dataframe:

In [10]:
cps_controls = pd.read_stata('../dw_data/cps_controls.dta')
cps_controls2 = pd.read_stata('../dw_data/cps_controls2.dta')
cps_controls3 = pd.read_stata('../dw_data/cps_controls3.dta')
psid_controls = pd.read_stata('../dw_data/psid_controls.dta')
psid_controls2 = pd.read_stata('../dw_data/psid_controls2.dta')
psid_controls3 = pd.read_stata('../dw_data/psid_controls3.dta')
nsw_dw = pd.read_stata('../dw_data/nsw_dw.dta')

In [11]:
# upon inspection we see that the experimental dataset (nsw_dw.dta) includes both treated and controlled individuals:
nsw_dw.treat.value_counts()

0.0    260
1.0    185
Name: treat, dtype: int64

### creating and defining covariates:

In [12]:
# This is a function that concatenates treatment and control data and adds all additional variables 
# as indicated in the footnote of Table 3 (Deheija & Wahba, 1999) to a dataset. 
# Those that are not needed are gonna be selected out later.

def add_variables(df):
    # concatenating treatment and control:
    control_dataset = pd.concat([nsw_dw, df], axis=0)
    control_dataset = control_dataset.reset_index(drop=True)

    # unemployment rate dummies:
    control_dataset['u74'] = control_dataset.re74.apply(lambda x: 1 if x == 0 else 0)
    control_dataset['u75'] = control_dataset.re75.apply(lambda x: 1 if x == 0 else 0)

    # higher order terms:
    control_dataset['education_2'] = control_dataset.education**2
    control_dataset['age_2'] = control_dataset.age**2
    control_dataset['age_3'] = control_dataset.age**3
    control_dataset['re74_2'] = control_dataset.re74**2
    control_dataset['re75_2'] = control_dataset.re75**2

    # interactions:
    control_dataset['education_re74'] = control_dataset.education*control_dataset.re74
    control_dataset['u74_black'] = control_dataset.u74*control_dataset.black
    
    return control_dataset

# applying the function to each comparison group separately:
cps_controls = add_variables(cps_controls)
cps_controls2 = add_variables(cps_controls2)
cps_controls3 = add_variables(cps_controls3)
psid_controls = add_variables(psid_controls)
psid_controls2 = add_variables(psid_controls2)
psid_controls3 = add_variables(psid_controls3)


In [13]:
# giving names to the datasets that will later help us use different covariates for each dataset:

cps_controls.name = 'cps1'
cps_controls2.name = 'cps2'
cps_controls3.name = 'cps3'
psid_controls.name = 'psid1'
psid_controls2.name = 'psid2'
psid_controls3.name = 'psid3'

In [14]:
# To compute propensity scores we define usage of covariates for different datasets:

covariates_cps = ['age', 'age_2', 
              'education', 'education_2', 'nodegree', 
              'married', 'black', 'hispanic', 're74', 're75',
              'u74', 'u75', 'education_re74', 'age_3']

covariates_psd1 = ['age', 'age_2', 
              'education', 'education_2', 'nodegree', 
              'married', 'black', 'hispanic', 're74', 're75', 're74_2', 're75_2',
              'u74_black']

covariates_psd23 = ['age', 'age_2', 
              'education', 'education_2', 'nodegree', 
              'married', 'black', 'hispanic', 're74', 're75', 're74_2', 're75_2',
              'u74', 'u75']


#### logistic regression to estimate propensity scores & nearest neighbor matching to obtain the ATT


In [15]:
# This function takes as an input a treated and a non-treated dataframe and creates 
# the counterfactual outcome of the treated units (Y0i) by matching (Nearest Neighbor) on propensity scores:
def get_matching_pairs(treated_df, non_treated_df):

    treated_x = treated_df.pscore.values
    non_treated_x = non_treated_df.pscore.values

    nbrs = NearestNeighbors(n_neighbors=1).fit(non_treated_x.reshape(-1, 1))
    distances, indices = nbrs.kneighbors(treated_x.reshape(-1, 1))
    indices = indices.reshape(indices.shape[0])
    matched = non_treated_df.loc[indices]
    return matched


#### logistic regression to estimate propensity scores: ####

# looping over each comparison dataset:
for control_dataset in [psid_controls, psid_controls2, psid_controls3, cps_controls, cps_controls2, cps_controls3]: 
    
    # use different covariates for each control group:
    if control_dataset.name == psid_controls.name:
        covariates = covariates_psd1
    elif control_dataset.name in [psid_controls2.name, psid_controls3.name]:
        covariates = covariates_psd23
    elif control_dataset.name in [cps_controls.name, cps_controls2.name, cps_controls3.name]:
        covariates = covariates_cps
        
    # define dependent and independent variables:
    y = control_dataset['treat']
    X = control_dataset[covariates]
    
    # option to drop nodegree as indicated in the problem set:
    control_dataset.drop('nodegree', axis=1, inplace=True)

    # create propensity scores with a logit model:
    logit = LogisticRegression(random_state=5, max_iter=1000)
    model = logit.fit(X, y)
    propensity = [n[1] for n in model.predict_proba(X)]
    control_dataset['pscore'] = propensity
    

#### perform nearest neighbor matching based on propensity scores: ####

    # The experimental controls are assigned to treatment as they are from the same distribution as treated
    # (Deheija & Wahba p. 1057 bottom left)
    treated_df = control_dataset[control_dataset.data_id=='Dehejia-Wahba Sample'].reset_index(drop=True)
    non_treated_df = control_dataset[control_dataset.data_id!='Dehejia-Wahba Sample'].reset_index(drop=True)
        
    # apply nearest neighbor matching function defined above:
    matched_df = get_matching_pairs(treated_df, non_treated_df)
    # assign propensity scores to the dataframe of the treated:
    treated_df['re78_cf'] = matched_df.re78.values
    
    # option to test whether pscore computation makes sense:
    #print(matched_df.pscore.values - treated_df.pscore.values < 0.05)
    
    #### compute att: ####
    # we select only the target group of interest (treated individuals of the experimental group):
    att = treated_df[treated_df['treat']==1]['re78'].mean() - treated_df[treated_df['treat']==1]['re78_cf'].mean()
    
    print(f'ATT for {control_dataset.name}: {att}')

ATT for psid1: 887.01953125
ATT for psid2: 1394.91748046875
ATT for psid3: 2596.96435546875
ATT for cps1: 927.732421875
ATT for cps2: -388.21875
ATT for cps3: 551.1279296875


In [8]:
#X = treated_df['pscore'].values
#y = treated_df['re78'].values
#X = sm.add_constant(X.ravel())
#
#results = sm.OLS(y,X).fit()
#print(f'coefficient: {results.params[0]}')
#print(f'standard error: {results.bse[0]}')